In [5]:
%matplotlib qt6
import csv
import cv2
import os

import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import npsam as ns
import tkinter as tk

from aicsimageio import AICSImage
from IPython.display import clear_output
from pathlib import Path
from PIL import Image, ImageTk, ImageEnhance, ImageFilter
from readlif.reader import LifFile
from skimage import exposure
from tkinter import filedialog, messagebox, ttk, simpledialog


def Param_GUI():
    """ Param_GUI() 
    Creation of the GUI for file selection and resolution flags.
    It allows the user to choose a file, select a resolution (512 or 2048),
    and returns the file path, folder path, and selected resolution flag.

    Parameters:
    -----------
        @None
    
    Output:
    -----------
        @file_path: path of the selected file
        @folder_path: path of the folder in which the file is located
        @selected_resolution: decide if images will be in original resolution (usually 2048x2048) or resized to 512x512
    """
    
    def file_selection():
        global file_path
        # Asking the file path
        file_path = filedialog.askopenfilename(title="Select a file")
        
        # Printing the file path and the folder path
        if file_path:
            folder_path = os.path.dirname(file_path)  # Extracting folder path from the file path
            file_label.config(text=f"File selected: {file_path}")
            folder_label.config(text=f"Folder: {folder_path}")
    
    def done_action():
        global selected_resolution
        # Closing the window
        root.destroy()
        return file_path, os.path.dirname(file_path), selected_resolution

    def resolution_choice(value):
        global selected_resolution
        selected_resolution = value

    # Creating the dialog window
    root = tk.Tk()
    root.title("File and Folder Selection")

    # Button for selecting the file
    file_button = tk.Button(root, text="Select file", command=file_selection)
    file_button.pack(padx=8, pady=8)

    # Labels to display selected file and folder paths
    file_label = tk.Label(root, text="No file selected")
    file_label.pack(padx=18, pady=8)
    folder_label = tk.Label(root, text="No folder selected")
    folder_label.pack(padx=18, pady=8)

    resolution_frame = tk.Frame(root)
    resolution_frame.pack(padx=8, pady=8)

    # Shared IntVar for radio buttons, default is 0 (512)
    resolution_var = tk.IntVar(value=0)

    resolution_512 = tk.Radiobutton(resolution_frame, text="Downsample to 512x512 pixel \n (Faster)", variable=resolution_var, value=0, command=lambda: resolution_choice(0)    )
    resolution_512.grid(row=0, column=0)

    resolution_2048 = tk.Radiobutton( resolution_frame, text="Original resolution \n (Usually is 2048x2048 pixel)", variable=resolution_var, value=1, command=lambda: resolution_choice(1))
    resolution_2048.grid(row=0, column=1)

    # Creating the "Done" button that will save and return the file and folder paths
    done_button = tk.Button(root, text="Done", command=done_action)
    done_button.pack(side=tk.BOTTOM, anchor="e", padx=8, pady=8)

    root.mainloop()

    return file_path, os.path.dirname(file_path), selected_resolution

def creating_subfolders(file_path, folder_path):
    """ creating_subfolder(folder path)
    The function creates a sub folder inside the folder path.

    Parameters:
    -----------
    @folder_path: path of the folder in which file is stored

    Output:
    -----------
    @subfolder_path_foci: path of the subfolder for "foci" data
    @subfolder_path_cell: path of the subfolder for "cell" data
    """
    # Initializing the file name, folder path and a counter for all the folders
    f_name = Path(file_path).stem
    f_path = Path(folder_path)
    counter = 1

    while True:
        # Constructing the folder name
        sub_folder = f_path / f"{f_name}_{counter:02d}"
        
        # If the folder doesn't exist, create it
        if not sub_folder.exists():
            sub_folder.mkdir(parents=True, exist_ok=True)
            
            # Create Foci subfolder
            subfolder_foci = sub_folder / "Foci"
            subfolder_foci.mkdir(exist_ok=True)
            
            # Create Cellule subfolder
            subfolder_cell = sub_folder / "Cellule"
            subfolder_cell.mkdir(exist_ok=True)
            
            return subfolder_foci, subfolder_cell
    
        counter += 1

def internal_folder(folder):
    """ internal_folder(folder)
    The function creates three separate folders inside the main one: one for the original images, one for the masks and one for the segmented images.

    Parameters:
    -----------
    @folder: Root folder

    Output:
    -----------
    @img_folder: Folder for images
    @mask_folder: Folder for masks
    @seg_folder: Folder for segmented images
    """
    
    img_folder = os.path.join(folder, "Images") # Creating path for images
    mask_folder = os.path.join(folder, "Cells") # Creating path for cells
    seg_folder = os.path.join(folder, "Overlay") # Creating path for the segmented images

    os.makedirs(img_folder, exist_ok=True) # Creating all the folders
    os.makedirs(mask_folder, exist_ok=True)
    os.makedirs(seg_folder, exist_ok=True)
    
    return img_folder, mask_folder, seg_folder

def brightness_adjustment(data, b_value):
    """ brightness_adjustment(data) -> b_value, image
    GUI for adjusting the brightness of the image. 
    Parameters:
    -----------
        @data: single channel 8-bit image.
        @b_value: value of brightness 
            If function is inside a for-loop, providing a not-zero b_value will directly adjust the image brightness
    
    Output:
    -----------
        @brightness_value : value of the brightness taken from the slider
                            Code extract the choosen brightness value from the GUI
                            
        @adjusted image: 8-bit image with brightness value adjusted
    """
    if b_value == 0:
        def adjust_brightness(image, value):
            #Adjust brightness of the image
            enhancer = ImageEnhance.Brightness(image)
            return enhancer.enhance(1 + value / 100)
        
        def update_image(value):
            #Update the displayed image based on slider value
            global original_image, initial_image, current_image
            value = float(value)
    
            # If slider is at zero, return to the original image, otherwise the image is modifyied
            if value == 0:
                current_image = initial_image
                
            else:
                current_image = adjust_brightness(initial_image, value)
                
    
            # Updating displayed image
            photo = ImageTk.PhotoImage(current_image)
            canvas.create_image(0, 0, image=photo, anchor="nw")
            canvas.image = photo
    
        def display_image(img):
            # Displaying the image
            pil_img = Image.fromarray(img)
            return pil_img
    
        def done_action():
            # Saving the brightness value from the slider and the adjusted image
            global brightness_value, current_image, adjusted_image
            # Get the values from the checkboxes when "Done" is clicked
            brightness_value = brightness_slider.get()
            adjusted_image = adjust_brightness(Image.fromarray(data), brightness_value)
            adjusted_image = np.array(adjusted_image)
            # Closing window
            root.destroy()
            
        
        # Create the main window
        root = tk.Tk()
        root.title("Image Brightness Adjuster")
        # Get the screen size (monitor resolution)
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
    
        # Resize the image to fit within the screen
        img_width, img_height = data.shape[1], data.shape[0]
        scale_factor = min(screen_width / img_width, screen_height / img_height, 1)
        new_width = int(img_width * scale_factor)
        new_height = int(img_height * scale_factor)

        # Labeling window
        global image_label
        image_label = tk.Label(root)
        image_label.pack(side="right", padx=10, pady=10)
        
        # Display the original image
        global original_image, initial_image, adjusted_image
        # Resize the image
        initial_image = Image.fromarray(data).resize((new_width, new_height), Image.Resampling.LANCZOS)
        original_image = initial_image  # Initially, they are the same

        # Create a frame for the canvas and sliders
        main_frame = tk.Frame(root)
        main_frame.pack(fill="both", expand=True)
    
        # Create a canvas to display the image
        canvas_frame = tk.Frame(main_frame)
        canvas_frame.pack(side="left", padx=10, pady=10, fill="both", expand=True)
    
        canvas = tk.Canvas(canvas_frame, width=new_width, height=new_height)
        canvas.pack(side="left", padx=0, pady=0, fill="both", expand=True)
    
        # Create a frame for the sliders
        right_frame = tk.Frame(main_frame)
        right_frame.pack(side="right", padx=10, pady=10, fill="y")

        # Display the initial image on canvas
        photo = ImageTk.PhotoImage(initial_image)
        canvas.create_image(0, 0, image=photo, anchor="nw")
        canvas.image = photo  # Keep a reference to avoid garbage collection
        
        # Create slider for brightness adjustment
        brightness_slider = tk.Scale(right_frame, from_=-100, to=1000, resolution=1, orient=tk.HORIZONTAL, label="Brightness", command=update_image)
        # Setting initial value to 0
        brightness_slider.set(0)
        brightness_slider.pack(pady=5)
    
        # "Done" button for closing and saving 
        done_button = tk.Button(right_frame, text= "Done", command = lambda: done_action())
        done_button.pack(padx=5, pady=5)
    
        
        root.mainloop()

        return brightness_value, adjusted_image

    else: 
        def adjust_brightness(image, value):
            #Adjust brightness of the image
            enhancer = ImageEnhance.Brightness(image)
            return enhancer.enhance(1 + value / 100)
            
        img = Image.fromarray(data)
        image = adjust_brightness(img, b_value)
        adjusted_image = np.array(image)
        return adjusted_image

def file_selection_gui():
    """file_selection_gui()
    Creates a GUI that allows user to decide if script has to analyse all the images or just some of them. If ALL IMAGES IN THE FOLDER flag is pressed, 
    script will analyse all of them. Otherwise, it will analyse those inserted in the text box. Use comme as separator and '-' (minus)
    to span. 

    Parameter:
    -----------
    None

    Output:
    -----------
    @flag: All images in the folder if pressed
    @text: Select the individual images
    """
    result = {"flag": 0, "text": ""}

    def on_done(): # Output obtained after pressing done button
        if all_var.get():
            result["flag"] = 1
            result["text"] = ""
        else:
            result["flag"] = 0
            result["text"] = entry_box.get()
        root.destroy()

    # Create the main window
    root = tk.Tk()
    root.title("Select the images to analyse:")
    root.geometry("400x150")

    # Create checkbox for "ALL IMAGES"
    all_var = tk.IntVar()
    all_checkbox = tk.Checkbutton(root, text="ALL IMAGES IN THE FOLDER", variable=all_var)
    all_checkbox.pack(anchor="w", padx=10, pady=5)

    # Label for the entry box
    label = tk.Label(root, text="Select the individual images. Use ',' to separate and '-' to span (e.g. 1, 2, 5-10)")
    label.pack(anchor="w", padx=10)
    # Create single-line entry box
    entry_box = tk.Entry(root, width=50)
    entry_box.pack(padx=10, pady=5)

    # Create the DONE button
    done_button = tk.Button(root, text="DONE", command=on_done)
    done_button.pack(anchor="se", padx=10, pady=10)

    root.mainloop()

    return result["flag"], result["text"]

def parse_number_string(s):
    """ parse_number_string(s)
    Transform a string into numbers. Use comma as separator and '-' (minus) to take all the numbers in between.

    Parameters:
    -----------
    @s: string

    Output:
    -----------
    @numbers: array containing all the numbers converted from the string
    """
    numbers = []
    parts = s.split(',')

    for part in parts:
        part = part.strip()
        if '-' in part:
            try:
                start, end = map(int, part.split('-'))
                numbers.extend(range(start, end + 1))
            except ValueError:
                raise ValueError(f"Invalid range format: {part}")
        else:
            try:
                numbers.append(int(part))
            except ValueError:
                raise ValueError(f"Invalid number format: {part}")
    
    return numbers

def cell_count(file_path, folder_path, selected_resolution):
    """ cell_count(file_path, folder_path) 
    The function takes as input parameters the path of the file and it's folder. The file MUST be a .lif file. 
    First, it opens the dataset and creates a list of images from it. Creates several subfolders for storing the results and then saves both foci 
    cellules in their respective folders. A for loop iterates over all the images and finds all the cellules inside, returning the mask and an image
    with the mask overlayed on the original image, both in their folders. Finally, it creates a .csv file with the number of the cellules in each image.

    Parameters:
    -----------
    @file_path: Path of the file
    @folder_path: Path in which the file is stored
    @selected_resolution: resolution of the images

    Output:
    -----------
    None
    """

    data_lif = LifFile(file_path) # Opening the file .lif
    scale_factor = round(AICSImage(file_path).physical_pixel_sizes[1], 3) # Extracting the rescaling factor from the metadata

    cell_folder = creating_subfolders(file_path, folder_path)[1] # Creating folder for results 
    cell_img_folder, cell_mask_folder, cell_seg_folder = internal_folder(cell_folder) # Creating all subfolders for cellules

    images = [i for i in data_lif.get_iter_image()] # Exporting the image 
    string_filenames = [] # Preparing a list for all the images
    flag = True
    cell_counted = []
    foci_counted = []

    # Creating the csv file
    csv_filename_B = os.path.join(cell_folder, "Cell counter - Cellule.csv")
    
    for i, image in enumerate(images):
        string_filenames.append(f'Image_{i+1}.png')

        if flag: 
            cell_data = cv2.normalize(np.array(image.get_frame(z=0, t=0, c=1)), 0, 65535, cv2.NORM_MINMAX)
            cell_8bit = np.clip((exposure.equalize_adapthist(cell_data) * 255).astype(np.uint8), 0, 255) # Moving from a 16bit image to a 8bit image

            if selected_resolution: # If the resolution is 2048x2048 does nothing, otherwise it resize the image
                pass
            else:
                cell_8bit = cv2.resize(cell_8bit, dsize = (512, 512), interpolation = cv2.INTER_AREA)

            cell_b_value, cell_img = brightness_adjustment(cell_8bit, 0) # Brightness adjusting
            
            flag = False

        else:
            cell_data = cv2.normalize(np.array(image.get_frame(z=0, t=0, c=1)), 0, 65535, cv2.NORM_MINMAX)
            cell_8bit = np.clip((exposure.equalize_adapthist(cell_data) * 255).astype(np.uint8), 0, 255) # Moving from a 16bit image to a 8bit image

            if selected_resolution: # If the resolution is 2048x2048 does nothing, otherwise it resize the image
                pass
            else:
                cell_8bit = cv2.resize(cell_8bit, dsize = (512, 512), interpolation = cv2.INTER_AREA)
                
            cell_img = brightness_adjustment(cell_8bit, cell_b_value) # Brightness adjusting

        cv2.imwrite(os.path.join(cell_img_folder, string_filenames[i]), cell_img)   # For saving cell images

    flag_all, text_images = file_selection_gui() # Allows to select which images will be analysed: ALL or some of them

    if flag_all: # If user select ALL IMAGES in the previous GUI
        for i, img in enumerate(os.listdir(cell_img_folder)): # Running through the images 
            img_path = os.path.join(cell_img_folder, string_filenames[i])
            clear_output(wait=True)
            print(f"Wait... Processing image: {i+1}/{len(os.listdir(cell_img_folder))}") 
            s = ns.NPSAM(img_path) # Starting segmentation
            s.set_scaling(str(scale_factor)) # Selecting scaling factor 
    
            s.segment(SAM_model = 'h', edge_filter = False) # Segmenting using the huge algorithm. It takes a lot of time but higher precsion 
    
            cell_counted.append(len(s.seg[0])) # Appending the counted cellules
            # Saving the mask 
            mask_img = s.seg[0].mean() # Creating the mask as a variable using hyperspy logic (since s is a Signal2D)
            plt.imsave(os.path.join(cell_mask_folder, string_filenames[i].replace("Image", "Cells")), mask_img, cmap = 'viridis') # Saving mask as png in his directory
            rescaled_mask_img = (((np.array(mask_img) - np.min(np.array(mask_img))) / (np.max(np.array(mask_img)) - np.min(np.array(mask_img)))) * 255).astype(np.uint8)
    
            # Overlapping mask to image
            mask_coloured = cv2.applyColorMap(rescaled_mask_img, cv2.COLORMAP_TURBO)
            rgb_image = cv2.cvtColor(np.array(cv2.imread(img_path)), cv2.COLOR_BGR2RGB) # RGB image 
            
            # Defining some parameters for the mask
            transparency = 0.6
            
            overlay = cv2.addWeighted(rgb_image, 1 - transparency, mask_coloured, transparency, 0)
            plt.imsave(os.path.join(cell_seg_folder, string_filenames[i].replace("Image", "Overlay")), overlay) # Saving overlayed image as png in his directory
    
            # Creating csv file
            if i == 0:  # Only write headers once (on the first iteration)
                with open(csv_filename_B, 'w', newline='') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    csv_writer.writerow(["File name", "Counted cells", "", "Brightness value"])
                
            with open(csv_filename_B, 'a', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                # Write data for each string_filenames and cell_number
                csv_writer.writerow([string_filenames[i], cell_counted[i], "", cell_b_value if i == 0 else ""])
    
    else: # If user want  to analyse only specific images
        list_images = parse_number_string(text_images) # Convert the string from the GUI in numbers
        iteration = 0 # Count the iterations and allows to correctly save the values in the csv file
        for i in list_images: # Running through the images 
            img_path = os.path.join(cell_img_folder, string_filenames[i-1])
            clear_output(wait=True)
            print(f"Wait... Processing image: {i}/{len(os.listdir(cell_img_folder))}") 
            s = ns.NPSAM(img_path) # Starting segmentation
            s.set_scaling(str(scale_factor)) # Selecting scaling factor 
    
            s.segment(SAM_model = 'h', edge_filter = False) # Segmenting using the huge algorithm. It takes a lot of time but higher precsion 
    
            cell_counted.append(len(s.seg[0])) # Appending the counted cellules
            # Saving the mask 
            mask_img = s.seg[0].mean() # Creating the mask as a variable using hyperspy logic (since s is a Signal2D)
            plt.imsave(os.path.join(cell_mask_folder, string_filenames[i-1].replace("Image", "Mask")), mask_img, cmap = 'viridis') # Saving mask as png in his directory
            rescaled_mask_img = (((np.array(mask_img) - np.min(np.array(mask_img))) / (np.max(np.array(mask_img)) - np.min(np.array(mask_img)))) * 255).astype(np.uint8)
    
            # Overlapping mask to image
            mask_coloured = cv2.applyColorMap(rescaled_mask_img, cv2.COLORMAP_TURBO)
            rgb_image = cv2.cvtColor(np.array(cv2.imread(img_path)), cv2.COLOR_BGR2RGB) # RGB image 
            
            # Defining some parameters for the mask
            transparency = 0.6
            
            overlay = cv2.addWeighted(rgb_image, 1 - transparency, mask_coloured, transparency, 0)
            plt.imsave(os.path.join(cell_seg_folder, string_filenames[i-1].replace("Image", "Overlay")), overlay) # Saving overlayed image as png in his directory

            
            # Creating csv file
            if iteration == 0:  # Only write headers once (on the first iteration)
                with open(csv_filename_B, 'w', newline='') as csvfile:
                    csv_writer = csv.writer(csvfile)
                    csv_writer.writerow(["File name", "Counted cells", "", "Brightness value"])
    
            with open(csv_filename_B, 'a', newline='') as csvfile:
                csv_writer = csv.writer(csvfile)
                # Write data for each string_filenames and cell_number
                csv_writer.writerow([string_filenames[i-1], cell_counted[iteration], "", cell_b_value if iteration == 0 else ""])
    
            iteration += 1
        
def Seg_cell_v4():
    """ Seg_cell_v4()
    Function that count the number of cellules in a .lif dataset
    """
    
    file_path, folder_path, selected_resolution = Param_GUI()
    cell_count(file_path, folder_path, selected_resolution)
    clear_output(wait=True)
    print("Count completed!")


In [ ]:
Seg_cell_v4()

Wait... Processing image: 4/61
